In [1]:
import numpy as np
import pandas as pd

In [2]:
pitching_df = pd.read_csv("db/pitching4.csv")

In [3]:
pitching_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 8 columns):
playerID      56 non-null object
G             56 non-null int64
W             56 non-null int64
SO            56 non-null int64
PCT           56 non-null float64
ERA           56 non-null float64
WHIP          56 non-null float64
HOF_member    56 non-null int64
dtypes: float64(3), int64(4), object(1)
memory usage: 3.6+ KB


In [4]:
X = pitching_df[["G","W","SO","PCT","ERA","WHIP"]]
y = pitching_df["HOF_member"]

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [6]:
# split data into test and train
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=27)


In [7]:
# Normalize the data.  Here, we are using StandardScaler
X_scaler = StandardScaler().fit(X_train)
#X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



C:\Users\Laptop 1\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """
C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [8]:
model = LogisticRegression(C=1.0, penalty='l2', random_state=0, solver="saga", max_iter=5000)

In [9]:
X_train_scaled.shape

(42, 6)

In [11]:
X_test_scaled.shape

(14, 6)

In [12]:
model.fit(X_train_scaled,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=5000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [13]:
from sklearn.metrics import mean_squared_error

In [14]:
predictions = model.predict(X_test_scaled)
MSE = mean_squared_error(y_test, predictions)
r2 = model.score(X_test_scaled, y_test)

print(f"MSE: {MSE}, R2: {r2}") 


MSE: 0.07142857142857142, R2: 0.9285714285714286


If curious about which players in the test split were predicted for what, we can take these two outputs, import and pair them

In [24]:
X_test["predictions"] = predictions
X_test.head()

C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,G,W,SO,PCT,ERA,WHIP,predictions
49,315,144,1905,0.595041,3.298059,1.178835,1
18,355,161,1024,0.540268,3.469466,1.353817,1
20,346,153,742,0.562500,3.614173,1.346027,1
6,392,192,914,0.600000,2.804982,1.228068,1
11,370,170,1004,0.584192,2.784510,1.240369,1


In [25]:
X_test.to_csv("db/Xtest.csv", index=False, header=True)

Here's what we've been waiting for:  Let's make Hall-Of-Fame predictions for more recent and current players.

In [15]:
recent_df = pd.read_csv("db/pitching_recent.csv")

In [16]:
recent_players = recent_df["playerID"]
X_recent = recent_df[["G","W","SO","PCT","ERA","WHIP"]]


In [17]:
X_recent_scaled = X_scaler.transform(X_recent)

C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [18]:
predictions_recent = model.predict(X_recent_scaled)

In [19]:
X_recent["predictions"] = predictions_recent
X_recent["playerID"] = recent_players

C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
people_df = pd.read_csv("db/people.csv")

In [21]:
players_df = people_df[["playerID","nameFirst","nameLast"]]
players_df["nameFull"] = players_df["nameFirst"]+' '+players_df["nameLast"]
players_df.head()

C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,playerID,nameFirst,nameLast,nameFull
0,aardsda01,David,Aardsma,David Aardsma
1,aaronha01,Hank,Aaron,Hank Aaron
2,aaronto01,Tommie,Aaron,Tommie Aaron
3,aasedo01,Don,Aase,Don Aase
4,abadan01,Andy,Abad,Andy Abad


In [22]:
X_recent.to_csv("db/Xrecent.csv", index=False, header=True)

In [23]:
Predicted_for_HOF = X_recent.loc[X_recent["predictions"] == 1, :]
Predicted_for_HOF = pd.merge(Predicted_for_HOF, players_df, on="playerID", how="left")
del Predicted_for_HOF["nameFirst"]
del Predicted_for_HOF["nameLast"]
display(Predicted_for_HOF)

,G,W,SO,PCT,ERA,WHIP,predictions,playerID,nameFull
0,331,144,1250,0.569170,3.839189,1.276577,1,buehrma01,Mark Buehrle
1,329,122,1682,0.575472,3.545649,1.223155,1,greinza01,Zack Greinke
2,292,143,1438,0.674528,3.192366,1.166412,1,hallaro01,Roy Halladay
3,307,121,1922,0.570755,3.311869,1.146664,1,hamelco01,Cole Hamels
4,331,136,1806,0.548387,3.704463,1.171897,1,harenda01,Dan Haren
5,322,139,2065,0.588983,3.132231,1.177686,1,hernafe02,Felix Hernandez
6,304,146,1372,0.654709,3.484303,1.253139,1,hudsoti01,Tim Hudson
7,292,144,2120,0.692308,2.362791,1.001550,1,kershcl01,Clayton Kershaw
8,295,128,1465,0.576577,4.101812,1.345416,1,lackejo01,John Lackey
9,304,136,1702,0.623853,3.519212,1.188970,1,leecl02,Cliff Lee
